In [2]:
from selenium.webdriver.remote.webelement import WebElement
from extraction import extract_data_from_page
from selenium.webdriver.common.by import By

In [3]:
def get_content(body: WebElement):
	content = ""
	for element in body.find_elements(By.CLASS_NAME, "article-body__paragraph"):
		content += element.text.strip() + "\n"
	return content

def get_author(body: WebElement):
	try:
		return body.find_element(By.CLASS_NAME, "article-body__byline__author").text.strip()
	except:
		return "not found"

def get_description(body: WebElement):
	try:
		return body.find_element(By.CLASS_NAME, "article-head__subtitle").text.strip()
	except:
		return "not found"


extract_data_from_page(
	input_file="../archive/pages_extracted/la_tercera/la_tercera_090725_cleanData.json", 
	output_file="test.json",
	get_author=get_author,
	get_description=get_description,
	get_content=get_content	
)

TypeError: extract_data_from_page() missing 1 required positional argument: 'limit_of_pages'